In [ ]:
# Iniciando o ambiente PySpark
import findspark
findspark.init()

In [ ]:
# Criando uma instância de SparkSession para iniciar uma sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .master('local[*]')\
    .appName('Conversão e manipulação dos arquivos CSV para Parquet')\
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [ ]:
PATH_PYSUS = r'data\001_PYSUS'
PATH_TABNET_UFMUN = r'data\002_TABNET\PA_UFMUN.csv'
PATH_TABNET_NIVCPL = r'data\002_TABNET\PA_NIVCPL.csv'
PATH_TABNET_CBOCOD = r'data\002_TABNET\PA_CBOCOD.csv'
PATH_TABNET_PROC_ID = r'data\002_TABNET\PA_PROC_ID.csv'
PATH_TABNET_G_PROC_ID = r'data\002_TABNET\PA_G_PROC_ID.csv'
PATH_TABNET_SG_PROC_ID = r'data\002_TABNET\PA_SG_PROC_ID.csv'

PATH_PYSUS_PARQUET = r'data\003_JOIN_DATASETS\PYSUS_PARQUET'
PATH_FINAL_DATASET = r'data\003_JOIN_DATASETS\FINAL_DATASET'

# Carregando DataFrame Spark

In [ ]:
def read_csv(path):
    return spark.read.csv(path, sep=';', header=True, inferSchema=True)

In [ ]:
%%time
df_pysus = read_csv(PATH_PYSUS)
df_pysus.printSchema()

In [ ]:
df_ufmun = read_csv(PATH_TABNET_UFMUN)
df_ufmun.printSchema()

In [ ]:
df_nivcpl = read_csv(PATH_TABNET_NIVCPL)
df_nivcpl.printSchema()

In [ ]:
df_cbocod = read_csv(PATH_TABNET_CBOCOD)
df_cbocod.printSchema()

In [ ]:
df_proc_id = read_csv(PATH_TABNET_PROC_ID)
df_proc_id.printSchema()

In [ ]:
df_g_proc_id = read_csv(PATH_TABNET_G_PROC_ID)
df_g_proc_id.printSchema()

In [ ]:
df_sg_proc_id = read_csv(PATH_TABNET_SG_PROC_ID)
df_sg_proc_id.printSchema()

# Convertendo base do PySUS de CSV para PARQUET para obter melhor desempenho de manipulação dos dados

In [ ]:
%%time
# 'partitionBy' especifica a coluna pela qual os dados serão particionados. Neste caso, os dados serão particionados pela coluna de data 'PA_CMP'
df_pysus.write.parquet(
        path=PATH_PYSUS_PARQUET,
        mode='overwrite',
        partitionBy='PA_CMP' )

In [ ]:
%%time
df_pysus = spark.read.parquet(PATH_PYSUS_PARQUET)
df_pysus.printSchema()

In [ ]:
f'O dataset possui {df_pysus.count()} linhas.'

# Juntando todos os DataFrames
O objetivo é adicionar as colunas de descrição obtidos no TabNet para enriquecer o dataset extraído da biblioteca PySUS

In [ ]:
df = df_pysus.join(df_ufmun, 'PA_UFMUN', how='left')
df.select('PA_UFMUN', 'DS_PA_UFMUN', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
df = df.join(df_nivcpl, 'PA_NIVCPL', how='left')
df.select('PA_NIVCPL', 'DS_PA_NIVCPL', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
df = df.join(df_cbocod, 'PA_CBOCOD', how='left')
df.select('PA_CBOCOD', 'DS_PA_CBOCOD', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
df = df.join(df_proc_id, 'PA_PROC_ID', how='left')
df.select('PA_PROC_ID', 'DS_PA_PROC_ID', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
df = df.join(df_g_proc_id, 'PA_G_PROC_ID', how='left')
df.select('PA_G_PROC_ID', 'DS_PA_G_PROC_ID', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
df = df.join(df_sg_proc_id, 'PA_SG_PROC_ID', how='left')
df.select('PA_SG_PROC_ID', 'DS_PA_SG_PROC_ID', 'PA_QTDAPR', 'PA_VALAPR')\
  .show(3, False)

In [ ]:
%%time
# Versão final do Dataset
df.write.parquet(
        path=PATH_FINAL_DATASET,
        mode='overwrite')

In [ ]:
spark.stop()